In [44]:
%pylab inline

import os, sys 
import pandas as pd
import seaborn as sns


Populating the interactive namespace from numpy and matplotlib


In [237]:
contigstats= pd.read_table('~/Desktop/Viruses/shHF4_coverage_stats.txt',index_col=0)

In [129]:
Quality= pd.read_table('/Users/silas/Desktop/Viruses/VIBRANT_results_shHF4_contigs/VIBRANT_genome_quality_shHF4_contigs.tsv',index_col=0)

Quality= Quality.loc[~ Quality.index.duplicated(keep=False) | (Quality.Quality=='complete circular')]
#Quality= Quality.query("(Quality=='high quality draft') | (Quality=='medium quality draft')")


In [141]:
Vs= pd.read_table('/Users/silas/Desktop/Viruses/VIBRANT_results_shHF4_contigs/VIBRANT_summary_results_shHF4_contigs.tsv', index_col=0)

In [45]:
orf=pd.read_csv("../WD/Genecatalog/clustering/orf2gene.tsv",
                     index_col=0,sep='\t')
orf.head()

orf['Contig']=orf.index.map(lambda s: '_'.join(s.split('_')[:-1]))
orf.head()

,Gene,Contig
ORF,,
shRTF8_0_1,Gene0000001,shRTF8_0
shRTF2_19_1,Gene0000001,shRTF2_19
shHF7_474_36,Gene0000001,shHF7_474
shRTC3_23_171,Gene0000001,shRTC3_23
shHF5_6_1,Gene0000001,shHF5_6


In [46]:
E=pd.read_table('../WD/Genecatalog/annotations/eggNog.tsv.gz',index_col=0)

In [47]:
orf=orf.join(E[['tax_scope','Taxonomy']],on='Gene')
orf=orf.fillna('Unknown')
nt=orf.groupby(['Contig','tax_scope']).size().unstack(fill_value=0)
nt= nt.reindex(contigstats.query('Length>1000').index).dropna()

In [48]:
Domain_of_interest='Viruses'

In [256]:
contigs_with_viruses =nt[Domain_of_interest]>0
nv= nt.loc[contigs_with_viruses].sort_values(Domain_of_interest)
#nv= nv.loc[(nv.sum(1)-nv.Viruses) <=nv.Viruses*1.5]
nv=nv.loc[:,~(nv==0).all()]


ratio = pd.Series(nv.Viruses/nv.Bacteria,name='VirusRatio').sort_values(ascending=False)


In [288]:
nt.loc['shHF4_5959']

tax_scope
Apicomplexa      0.0
Archaea          0.0
Arthropoda       0.0
Bacteria         2.0
Eukaryota        0.0
Fungi            0.0
Mammalia         0.0
Metazoa          0.0
Nematoda         0.0
Opisthokonta     0.0
Unknown          6.0
Viridiplantae    0.0
Viruses          0.0
Name: shHF4_5959, dtype: float64

In [279]:
ratio.loc[ratio>=0.8]

#ID
shHF4_16190         inf
shHF4_6581          inf
shHF4_28284         inf
shHF4_3536          inf
shHF4_13912         inf
shHF4_13725         inf
shHF4_18354         inf
shHF4_18522         inf
shHF4_26823         inf
shHF4_8328          inf
shHF4_25802         inf
shHF4_24351         inf
shHF4_18714         inf
shHF4_8928          inf
shHF4_1379     3.500000
shHF4_2579     2.500000
shHF4_1735     1.333333
shHF4_8855     1.000000
shHF4_25910    1.000000
shHF4_27312    1.000000
shHF4_22832    1.000000
shHF4_20398    1.000000
shHF4_20121    1.000000
shHF4_19710    1.000000
shHF4_18927    1.000000
shHF4_18480    1.000000
shHF4_4852     1.000000
shHF4_11067    1.000000
shHF4_15301    1.000000
shHF4_11712    1.000000
shHF4_26215    1.000000
shHF4_11781    1.000000
shHF4_8863     1.000000
shHF4_8608     1.000000
Name: VirusRatio, dtype: float64

In [293]:
coi='shHF4_28284'

In [294]:
display(nv.loc[coi])
print()
display(Quality.loc[coi])
print()
display(Vs.loc[coi])

tax_scope
Archaea          0.0
Bacteria         0.0
Eukaryota        0.0
Fungi            0.0
Metazoa          0.0
Nematoda         0.0
Unknown          2.0
Viridiplantae    0.0
Viruses          1.0
Name: shHF4_28284, dtype: float64

KeyError: 'shHF4_28284'

In [284]:
Quality.join(ratio).query('Quality!="low quality draft"')

,type,Quality,VirusRatio
scaffold,,,
shHF4_1876,lytic,medium quality draft,NaN
shHF4_1379,lytic,high quality draft,3.5
shHF4_285_fragment_3,lysogenic,medium quality draft,NaN
shHF4_106_fragment_1,lysogenic,high quality draft,NaN
shHF4_62_fragment_1,lysogenic,high quality draft,NaN
...,...,...,...
shHF4_1429,lytic,complete circular,NaN
shHF4_5959,lytic,complete circular,NaN
shHF4_2016,lytic,complete circular,NaN


## Bins

In [151]:
Bins= pd.read_table('/Users/silas/Desktop/Viruses/shHF4_viralmetabat.tsv',index_col=0,squeeze=True,header=None)
Bins.name='Bins'
Bins=Bins.loc[Bins!=0]

In [167]:
Quality.groupby([Bins,Quality.type]).size().unstack()

type,lysogenic,lytic
Bins,,
2.0,1.0,NaN
4.0,20.0,35.0
7.0,NaN,3.0
9.0,NaN,3.0
12.0,NaN,4.0
...,...,...
851.0,NaN,1.0
868.0,1.0,NaN
892.0,1.0,NaN
